In [1]:
#| label: app:interactive_sns_fig4

# %matplotlib inline

from src.cdk.analysis.cytosol import platereader as pr
import pandas as pd

import seaborn as sns
import seaborn.objects as so

import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

import src.cdk.logging

pr.plot_setup()

# load data and calculat SS values
data, platemap = pr.load_platereader_data("tests/test_data/cytation_dna_sweep.txt", "tests/test_data/platemap.csv")
kinetics = pr.kinetic_analysis(data);

def interactive_kinetic_by_experiment(my_data, my_kinetics, toggle_col="Experiment"):
    
    # Create widgets for the parameters
    toggle = widgets.ToggleButtons(
        options=my_data[toggle_col].unique().tolist(),
        description=toggle_col+":",
        disabled=False,
        button_style=''
    )

    # Define the first dropdown menu (level 0)
    level_zero_dropdown = widgets.Dropdown(
        options=['Velocity', 'Lag', 'Steady State', 'Fit'],
        value='Lag',
        description='Property:',
    )

    # Define the second dropdown menu (level 1)
    level_one_dropdown = widgets.Dropdown(
        options=[],  # Initially empty
        value=None,
        description='Item:',
    )

    # Define a plot placeholder
    plot_output = Output()

    # Function to update the second dropdown based on the first dropdown's selection
    def update_items(change):
        if change['new'] == 'Velocity':
            level_one_dropdown.options = kinetics.columns[kinetics.columns.get_level_values(0) == 'Velocity'].get_level_values(1).tolist()
        elif change['new'] == 'Lag':
            level_one_dropdown.options = kinetics.columns[kinetics.columns.get_level_values(0) == 'Lag'].get_level_values(1).tolist()
        elif change['new'] == 'Steady State':
            level_one_dropdown.options = kinetics.columns[kinetics.columns.get_level_values(0) == 'Steady State'].get_level_values(1).tolist()
        elif change['new'] == 'Fit':
            level_one_dropdown.options = kinetics.columns[kinetics.columns.get_level_values(0) == 'Fit'].get_level_values(1).tolist()
        
        # Also reset the second dropdown value to None each time first dropdown changes
        level_one_dropdown.value = None

    # Function to update the plot based on selected dropdown values
    def update_plot(change):
        # Extract the selected level 0 and level 1 values
        experiment = toggle.value
        level_0 = level_zero_dropdown.value
        level_1 = level_one_dropdown.value
        
        if level_1 is None:
            return  # If no item is selected, do nothing
        
        # Create a new plot based on the selections
        with plot_output:

            # Clear previous output (including the figure)
            plot_output.clear_output(wait=True)  # This will clear the previous plot

            plt.close()
            
            # Plotting based on selected dropdown values
            plt.figure(figsize=(10, 6))  # You can adjust the size of the plot

            #filter kinetics by experiment

            combined_df = pd.merge(my_kinetics[level_0], data[['Well', 'Name', 'Experiment']], on='Well', how='left').drop_duplicates()
            filter_combined = combined_df[combined_df['Experiment'] == experiment]

            sns.catplot(data=filter_combined, kind="bar", x="Name", y=level_1, height=5, aspect=1)
            plt.suptitle(f'{level_0} - {level_1} Plot', fontsize=16)
            plt.xticks(rotation=60, ha='right', fontsize=10)
            plt.tight_layout

            plt.show()

    # Link the first dropdown to update the second dropdown options
    level_zero_dropdown.observe(update_items, names='value')

    # Link the second dropdown to the plot update function
    level_one_dropdown.observe(update_plot, names='value')

    toggle.observe(update_plot, names='value')

    # Display both dropdowns and the plot
    display(toggle, level_zero_dropdown, level_one_dropdown, plot_output)

    # Call the update_items once to initialize the second dropdown options
    update_items({'new': level_zero_dropdown.value})

interactive_kinetic_by_experiment(data, kinetics, toggle_col="Experiment")

Failed to solve: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
Failed to solve: Optimal parameters not found: Number of calls to function has reached maxfev = 800.


ToggleButtons(description='Experiment:', options=('Concentration', 'Artifact'), value='Concentration')

Dropdown(description='Property:', index=1, options=('Velocity', 'Lag', 'Steady State', 'Fit'), value='Lag')

Dropdown(description='Item:', options=(), value=None)

Output()